#### import정리하기

In [59]:
import pandas as pd
import os, sys
import numpy as np
import warnings
import matplotlib.pyplot as plt
import re
import squarify
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from spacy.tokenizer import Tokenizer
from konlpy.tag import Okt
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings('ignore')

#### 그래프 한글 깨짐 방지

In [44]:
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=FutureWarning)

plt.style.use("ggplot")
plt.rc('font', family='Malgun Gothic')

#### 데이터 불러오기

In [45]:
df = pd.read_csv('동화데이터_최종.csv')

In [46]:
df.head()

,제목,내용,카테고리,키워드,테마
0,고양이와 쥐의 파트너십(동반자관계),쥐를 친구로 된 어떤 고양이가 살고 있었다. \r\n고양이는 자신이 쥐에 대해 사랑...,그림형제동화,"['고양이', '돼지 비계', '이름', '항아리', '세례', '대모', '친구'...",교회
1,건달들의 무리,하루는 수탉이 암탉에게 말했다. \r\n“나무열매들이 익어 가는 계절이 왔는데 슬슬...,그림형제동화,"['수탉', '암탉', '오리', '나그네', '주막 주인', '마차']",숲속
2,실 잣는 세 여인네들,옛날 옛적에 게을러서 실을 잣지 않으려는 소녀가 한 명 있었어요.\r\n엄마가 뭐라...,그림형제동화,"['소녀', '엄마', '왕비', '게으름', '세여인']",왕국
3,"지푸라기, 숯, 콩",옛날 어느 마을에 가난한 노파가 한 명 살고 있었어요.\r\n그녀는 접시에 콩들을 ...,그림형제동화,"['숯', '지푸라기', '콩', '탈출', '재봉사', '도움']",마을
4,"생쥐, 새, 소시지",옛날 어느 날 생쥐와 새와 ‘소시지’가 동무가 되어 한 집에서 살게 되었어요.\r\...,그림형제동화,"['소시지', '생쥐', '새', '뗄감', '요리', '후회']",숲속


### tokenize_test 함수로 토큰화 후 명사만 저장하기

In [47]:
# Okt 객체 생성
okt = Okt()

def tokenize_test(text):
    # 줄바꿈 제거
    text = text.replace("\n", " ")

    # 한글, 숫자, 공백을 제외한 모든 문자 제거
    replace = ""
    pattern = "[^가-힣0-9 ]"
    doc = re.sub(pattern, replace, text)

    tokens = []

    # 내용 중에서 명사만 tokens 변수에 저장하기
    for word in okt.nouns(doc):
        tokens.append(word)
    return tokens

In [48]:
df['Tokens'] = df['내용'].apply(tokenize_test)
df['Tokens'].head()

0    [쥐, 친구, 고양이, 살, 고양이, 자신, 쥐, 대해, 사랑, 우정, 말, 쥐, ...
1    [하루, 수탉, 암탉, 말, 나무, 열매, 계절, 슬슬, 언덕, 배, 잔뜩, 다람쥐...
2    [옛날, 옛적, 실, 잣, 소녀, 명, 엄마, 소녀, 꿈쩍, 안, 엄마, 화가, 나...
3    [옛날, 마을, 노파, 명, 살, 그녀, 접시, 콩, 요리, 해, 불, 활활, 다음...
4    [옛날, 날, 생쥐, 새, 소시지, 동무, 집, 살, 서로, 의지, 재산, 새, 일...
Name: Tokens, dtype: object

In [50]:
tfidf = TfidfVectorizer(tokenizer = tokenize_test,
                       ngram_range = (1,2),
                       max_features = 5000,
                       token_pattern = None)

dtm = tfidf.fit_transform(df['내용'])
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names_out())

In [51]:
dtm.head()

가게   가격        가기  가까스로  가까스로 탈출  가까이   가끔   가나   가난  가다가        가도  \
0  0.0  0.0  0.000000   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.020994   
1  0.0  0.0  0.051109   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
2  0.0  0.0  0.000000   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
3  0.0  0.0  0.000000   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
4  0.0  0.0  0.000000   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.000000   

         가득   가라   가로   가루   가마   가면  가면 길   가뭄   가선   가슴   가시  가시 네  가시나무  \
0  0.027775  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0   0.0   
1  0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0   0.0   
2  0.032381  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0   0.0   
3  0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0   0.0   
4  0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0   0.0   

    가야  가야 때   가요       가운데   가위   가을  가자미  가자미 망둑어   가장  가장 것  가장 새  가장 힘  \
0  0.0   0.0  0.0  0.000000  0.0  0.0  0.0      0.0  0.0   0.0   0.0   0.0   
1  0.0   0.0  0.0  0.000000  0.0  0.0  0.0      0.0  0.0   0.0   0.0   0.0   
2  0.0   0.0  0.0  0.000000  0.0  0.0  0.0      0.0  0.0   0.0   0.0   0.0   
3  0.0   0.0  0.0  0.071193  0.0  0.0  0.0      0.0  0.0   0.0   0.0   0.0   
4  0.0   0.0  0.0  0.000000  0.0  0.0  0.0      0.0  0.0   0.0   0.0   0.0   

    가정   가족   가죽  가죽 상인        가지  가지 간  가지 것  가지 고양이  가지 나  가지 말  가지 질문  \
0  0.0  0.0  0.0    0.0  0.000000   0.0   0.0     0.0   0.0   0.0    0.0   
1  0.0  0.0  0.0    0.0  0.000000   0.0   0.0     0.0   0.0   0.0    0.0   
2  0.0  0.0  0.0    0.0  0.000000   0.0   0.0     0.0   0.0   0.0    0.0   
3  0.0  0.0  0.0    0.0  0.000000   0.0   0.0     0.0   0.0   0.0    0.0   
4  0.0  0.0  0.0    0.0  0.022073   0.0   0.0     0.0   0.0   0.0    0.0   

   가지 집   가짜  가짜 도령   가축  가축 우리   가치  가파른   각각   각시        각자  각자 자신  \
0   0.0  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.000000    0.0   
1   0.0  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.000000    0.0   
2   0.0  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.000000    0.0   
3   0.0  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.000000    0.0   
4   0.0  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.077432    0.0   

          간   간담   간청   갈고   갈기  갈기 시작  갈까마귀  갈까마귀 자신   갈대        갈색  갈수록  \
0  0.000000  0.0  0.0  0.0  0.0    0.0   0.0      0.0  0.0  0.021848  0.0   
1  0.019955  0.0  0.0  0.0  0.0    0.0   0.0      0.0  0.0  0.000000  0.0   
2  0.000000  0.0  0.0  0.0  0.0    0.0   0.0      0.0  0.0  0.000000  0.0   
3  0.000000  0.0  0.0  0.0  0.0    0.0   0.0      0.0  0.0  0.000000  0.0   
4  0.034031  0.0  0.0  0.0  0.0    0.0   0.0      0.0  0.0  0.000000  0.0   

    갈증    감   감고   감독  감독 관   감동        감사   감옥   감탄       감히  갑자기    값  값 냥  \
0  0.0  0.0  0.0  0.0   0.0  0.0  0.000000  0.0  0.0  0.00000  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0   0.0  0.0  0.000000  0.0  0.0  0.02308  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0   0.0  0.0  0.000000  0.0  0.0  0.00000  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0   0.0  0.0  0.043107  0.0  0.0  0.00000  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0   0.0  0.0  0.000000  0.0  0.0  0.00000  0.0  0.0  0.0   

     갓   갓난    강  강 부모  강 한가운데   강가        강도   강둑   강물  강아지  강아지 마리  강아지 씨  \
0  0.0  0.0  0.0   0.0     0.0  0.0  0.000000  0.0  0.0  0.0     0.0    0.0   
1  0.0  0.0  0.0   0.0     0.0  0.0  0.025554  0.0  0.0  0.0     0.0    0.0   
2  0.0  0.0  0.0   0.0     0.0  0.0  0.000000  0.0  0.0  0.0     0.0    0.0   
3  0.0  0.0  0.0   0.0     0.0  0.0  0.000000  0.0  0.0  0.0     0.0    0.0   
4  0.0  0.0  0.0   0.0     0.0  0.0  0.043580  0.0  0.0  0.0     0.0    0.0   

   강아지 호주머니   강풍         개  개 고양이  개 그  개 대답  개 때     개 마리       개 말  개 모두  \
0       0.0  0.0  0.000000    0.0  0.0   0.0  0.0  0.00000  0.000000   0.0   
1       0.0  0.0  0.000000    0.0  0.0   0.0  0.0  0.00000  0.000000   0.0   
2       0.0  0.0  0.023982    0.0  0.0   0.0  0.0  0.00000  0.000000   0

In [52]:
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree') # 최근접 5이웃
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree')

In [53]:
result = nn.kneighbors([dtm.iloc[0]]) # 특정 행과 가장 유사한 문서 5개를 찾는다.
result

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


(array([[0.        , 1.19955372, 1.20928982, 1.2193616 , 1.24157924]]),
 array([[  0,  21, 204,  18, 238]], dtype=int64))

In [54]:
df.iloc[result[1][0]]

,제목,내용,카테고리,키워드,테마,Tokens
0,고양이와 쥐의 파트너십(동반자관계),쥐를 친구로 된 어떤 고양이가 살고 있었다. \r\n고양이는 자신이 쥐에 대해 사랑...,그림형제동화,"['고양이', '돼지 비계', '이름', '항아리', '세례', '대모', '친구'...",교회,"[쥐, 친구, 고양이, 살, 고양이, 자신, 쥐, 대해, 사랑, 우정, 말, 쥐, ..."
21,여우와 고양이,우연히 고양이가 숲에서 여우를 만났어요.\r\n고양이가 속으로 생각했어요.\r\n“...,그림형제동화,"['여우', '고양이', '오만함']",숲속,"[고양이, 숲, 여우, 고양이, 속, 생각, 그, 경험, 세상, 고양이, 애교, 목..."
204,고양이와 생쥐들,어떤 집이 생쥐들로 넘쳐나고 있었어요.\r\n소문을 듣고 고양이 한 마리가 냉큼 그...,이솝우화,"['생쥐', '고양이', '소문', '꾀']",집,"[집, 생쥐, 소문, 고양이, 마리, 냉큼, 그, 집, 살림, 다음, 마리, 차례차..."
18,세 행운아 (삼 형제의 행운),"아빠가 한 번은 세 아들을 자기 앞으로 불러, 첫째에게는 수탉 한 마리를, 둘째에게...",그림형제동화,"['아빠', '아들', '고양이', '왕', '궁전']",마을,"[아빠, 번은, 세, 아들, 자기, 앞, 첫째, 수탉, 마리, 둘째, 낫, 자루, ..."
238,독수리와 고양이와 암멧돼지,높은 떡갈나무 꼭대기에 독수리가 둥지를 만들었어요.\r\n그리고 그 나무줄기 중간에...,이솝우화,"['독수리', '고양이', '멧돼지', '교활함']",숲속,"[떡갈나무, 꼭대기, 독수리, 둥지, 그, 나무, 줄기, 중간, 고양이, 보금자리,..."


In [64]:
df['제목'].iloc[result[1][0][4]]

'독수리와 고양이와 암멧돼지'

### 하나의 함수로 동화 추천 목록 가져오기

In [67]:
# 함수 토큰화 및 명사 저장 함수
def tokenize_test(text):
    # 줄바꿈 제거
    text = text.replace("\n", " ")

    # 한글, 숫자, 공백을 제외한 모든 문자 제거
    replace = ""
    pattern = "[^가-힣0-9 ]"
    doc = re.sub(pattern, replace, text)

    tokens = []

    # 내용 중에서 명사만 tokens 변수에 저장하기
    for word in okt.nouns(doc):
        tokens.append(word)
    return tokens


# 유사한 동화 제목을 추천하여 csv파일로 저장하는 코드
def tale_recommendation():
    tfidf = TfidfVectorizer(tokenizer = tokenize_test,
                       ngram_range = (1,2),
                       max_features = 5000,
                       token_pattern = None)

    dtm = tfidf.fit_transform(df['내용'])
    dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names_out())

    nn = NearestNeighbors(n_neighbors=6, algorithm='kd_tree') # 최근접 5이웃
    nn.fit(dtm)
    
    # result = nn.kneighbors([dtm.iloc[1]]) # 특정 행과 가장 유사한 문서 5개를 찾는다.
    # title = df['제목'].iloc[result[1][0]]
    
    token_list = []
    # 문서를 자동으로 추가하기
    for i in range(len(df)):
        result = nn.kneighbors([dtm.iloc[i]]) # 특정 행과 가장 유사한 문서 5개를 찾는다.
        one_title = []
        for j in range(1, 6):
            one_title.append(df['제목'].iloc[result[1][0][j]])
        token_list.append(one_title)
        
    df['비슷한 동화'] = token_list
        
    return df


def main():
    # csv 불러오기
    df = pd.read_csv('동화데이터_최종.csv')
    okt = Okt()

    # 토큰화 내용을 Tokens 열에 추가하기
    df['Tokens'] = df['내용'].apply(tokenize_test)
    df = tale_recommendation()

if __name__ == "__main__":
    main()

In [68]:
df.head()

,제목,내용,카테고리,키워드,테마,Tokens,비슷한 동화
0,고양이와 쥐의 파트너십(동반자관계),쥐를 친구로 된 어떤 고양이가 살고 있었다. \r\n고양이는 자신이 쥐에 대해 사랑...,그림형제동화,"['고양이', '돼지 비계', '이름', '항아리', '세례', '대모', '친구'...",교회,"[쥐, 친구, 고양이, 살, 고양이, 자신, 쥐, 대해, 사랑, 우정, 말, 쥐, ...","[여우와 고양이, 고양이와 생쥐들, 세 행운아 (삼 형제의 행운), 독수리와 고양이..."
1,건달들의 무리,하루는 수탉이 암탉에게 말했다. \r\n“나무열매들이 익어 가는 계절이 왔는데 슬슬...,그림형제동화,"['수탉', '암탉', '오리', '나그네', '주막 주인', '마차']",숲속,"[하루, 수탉, 암탉, 말, 나무, 열매, 계절, 슬슬, 언덕, 배, 잔뜩, 다람쥐...","[코르베스 씨, 개와 수탉과 여우, 좁쌀 한 톨, 싸움닭들과 독수리, 도둑들과 수탉]"
2,실 잣는 세 여인네들,옛날 옛적에 게을러서 실을 잣지 않으려는 소녀가 한 명 있었어요.\r\n엄마가 뭐라...,그림형제동화,"['소녀', '엄마', '왕비', '게으름', '세여인']",왕국,"[옛날, 옛적, 실, 잣, 소녀, 명, 엄마, 소녀, 꿈쩍, 안, 엄마, 화가, 나...","[아마 부스러기 (부지런한 하녀), 게으른 실 잣는 여인, 트루데 아주머니, 숲 속..."
3,"지푸라기, 숯, 콩",옛날 어느 마을에 가난한 노파가 한 명 살고 있었어요.\r\n그녀는 접시에 콩들을 ...,그림형제동화,"['숯', '지푸라기', '콩', '탈출', '재봉사', '도움']",마을,"[옛날, 마을, 노파, 명, 살, 그녀, 접시, 콩, 요리, 해, 불, 활활, 다음...","[닭장의 홰(나무 막대), 숲 속의 노파, 어린 양과 작은 물고기, 말라깽이 리제,..."
4,"생쥐, 새, 소시지",옛날 어느 날 생쥐와 새와 ‘소시지’가 동무가 되어 한 집에서 살게 되었어요.\r\...,그림형제동화,"['소시지', '생쥐', '새', '뗄감', '요리', '후회']",숲속,"[옛날, 날, 생쥐, 새, 소시지, 동무, 집, 살, 서로, 의지, 재산, 새, 일...","[어리석은 소원, 새사냥꾼과 자고새와 수탉, 생쥐와 도사, 허영심 강한 갈까마귀, ..."


### 동화데이터_최종.csv 파일에 [추천 동화 목록] 열을 새로 생성하여 추천 동화 목록 저장하기

In [69]:
df.to_csv('수정된 동화.csv')